<a href="https://colab.research.google.com/github/jimkc/Scripts/blob/main/Collect_YouTubeComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Output as Pandas DataFrame

In [ ]:
import googleapiclient.discovery
import pandas as pd
import re

# Function to extract video ID from YouTube URL
def extract_video_id(url):
    # Regular expression for extracting the video ID
    regex = r"(?:v=|\/)([0-9A-Za-z_-]{11}).*"
    matches = re.search(regex, url)
    if matches:
        return matches.group(1)
    else:
        raise ValueError("Unable to extract Video ID from the provided URL.")

# Sample YouTube URL (User should replace this with the actual URL)
youtube_url = ""

# Set the parameter of developer key and how many comments to get below
DEVELOPER_KEY = ""
COMMENTS_TO_GET = 100
# Extracting the video ID from the URL
VIDEO_ID = extract_video_id(youtube_url)

api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part="snippet",
    videoId=VIDEO_ID,
    maxResults=COMMENTS_TO_GET
)
response = request.execute()

comments = []

for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    comments.append([
        comment['authorDisplayName'],
        comment['publishedAt'],
        comment['updatedAt'],
        comment['likeCount'],
        comment['textDisplay']
    ])

df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])

df.head(COMMENTS_TO_GET)


In [ ]:
import os

# Define the directory path
directory_path = '/mnt/data'

# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [ ]:
df.head(COMMENTS_TO_GET).to_csv('/mnt/data/my_dataframe.csv', index=False)